## TRABAJO FINAL METODOLOGÍA DE LA OPINIÓN PÚBLICA

### "Análisis de resultados en las elecciones generales 2023 para CABA"


#### Grupo: Matías Chiampan, Nicolas Faccini, Camila Valeff

El objetivo del siguiente trabajo consistió en examinar los resultados electorales en la instancia de elecciones generales dentro de CABA. 
Para ello se utilizó Python y diferentes librerias importadas con el fin de robustecer la capacidad de análisis, incorporando diversidad de gráficas. 

Las librerías se importaron a partir de las funciones "import" e "install", los scripts para este paso fueron los siguientes:

## Importación de librerías

In [ ]:
pip install contextily

: 

In [ ]:
pip install geopandas

: 

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import plotly.express as px

: 

Luego de importar las librerias requerida, importamos los archivos .csv que son nuestra fuente de información. Tenemos dos tipos de archivos: los referidos a los resultados de las elecciones en 2023 y las bases de datos del CENSO 2010 en Argentina. Estas últimas nos permitirán contar con variables sociodemográficos para las comunas de CABA, con el fin de explorar si existe algún patrón entre éstas y las elecciones de los votantes

## Bases de datos elecciones generales 2023

In [ ]:
df_resultados = pd.read_csv('2023_Generales\ResultadosElectorales_2023.csv')
df_caba = pd.read_csv('2023_Generales\CABA-circuitos-electorales.csv')
df_ambitos = pd.read_csv('2023_Generales/Ambitos_Generales_2023.csv')
df_colores = pd.read_csv('2023_Generales/Colores_2023.csv')

: 

#### Las tablas importadas cuentan con la siguiente estructura

In [ ]:
print("Tabla resultados")
print(f"Estructura: {df_resultados.shape}")
print("\n")
df_resultados.head().T

: 

In [ ]:
print("Circuitos electorales en CABA")
print(f"Estructura: {df_caba.shape}")
print("\n")
df_caba.head()

: 

In [ ]:
print("Ambitos Generales")
print(f'Estructura: {df_ambitos.shape}')
print("\n")
df_ambitos.head(15)

: 

## Bases de datos CENSO 2010

Las bases de datos del censo utilizadas son 8, siendo la base de viviendas, la de hogares y la de personas las 3 más importantes para el análisis del perfil sociodemográfico de las comunas en CABA. Las otras tablas se utilizarán para datos geográficos de la población, los cuales nos permiten relacionar la información del censo con las elecciones generales.

In [ ]:
Cen_viviendas = pd.read_csv('censo2010/censo/vivienda.csv')
Cen_hogares = pd.read_csv('censo2010/censo/hogar.csv')
Cen_personas = pd.read_csv('censo2010/censo/persona.csv')
Cen_dpto = pd.read_csv('censo2010/censo/dpto.csv')
Cen_frac = pd.read_csv('censo2010/censo/frac.csv')
Cen_prov = pd.read_csv('censo2010/censo/prov.csv')
Cen_radio= pd.read_csv('censo2010/censo/radio.csv')

: 

Para trabajar con las bases del censo 2010 se requirió generar un diagrama de entidad relación (DER) que permitiera un mayor entendimiento de sus datos. Cabe aclarar que éstos son centralmente códigos cuyas etiquetas se encuentran en la carpeta "lebels".
El DER se desarrolló de la siguiente forma:

<img src="DER_Censo2010.png">

#### Las tablas del CENSO 2010

Las tablas del censo 2010 se imprimen de la siguiente forma:

In [ ]:
print("Tabla de características de las viviendas")
print(f'Estructura: {Cen_viviendas.shape}')
Cen_viviendas.head()

: 

In [ ]:
print("Tabla características de los hogares")
print(f'Estructura: {Cen_hogares.shape}')
Cen_hogares.head()

: 

In [ ]:
print("Tabla de características de las personas")
print(f'Estructura: {Cen_personas.shape}')
Cen_personas.head()

: 

#### Merge Base de datos CENSO 2010

In [ ]:
#merge para unificar personas-hogares

merge_persona_hogar = pd.merge(Cen_personas, Cen_hogares[['HOGAR_REF_ID', 'ALGUNBI', 'VIVIENDA_REF_ID']], on='HOGAR_REF_ID', how='left')

#merge para unificar el combinado anterior con viviendas

merge_total = pd.merge(merge_persona_hogar, Cen_viviendas[['VIVIENDA_REF_ID', 'MUNI', 'LOCAL']], on='VIVIENDA_REF_ID', how='left')

#total base censo por persona, filtrado por las comunas que pertenecen a CABA.
censo_caba = merge_total[(merge_total['MUNI'] >= 20010001) & (merge_total['MUNI'] <= 20150001)]

censo_caba

: 

In [ ]:
#hacemos un replace para que los valores de id local coincidan con los ids de comuna en la base de elecciones (representan lo mismo)
nuevos_codigos = {2001010:1,
2002010:2,
2003010:3,
2004010:4,
2005010:5,
2006010:6,
2007010:7,
2008010:8,
2009010:9,
2010010:10,
2011010:11,
2012010:12,
2013010:13,
2014010:14,
2015010:15
}

censo_caba['LOCAL'] = censo_caba['LOCAL'].replace(nuevos_codigos)
censo_caba

: 

### Reestructuración de las tablas para caracterización de comunas

Con el fin de adaptar los resultados del censo a las necesidades de este trabajo, creamos nuevas tablas que nos permitirán luego conectar esta información con las bases de datos de los resultados electorales. Siguiente este propósito, nos hicimos las siguientes preguntas que orientarán más tarde el análisis:

    1. ¿Cuáles son las comunas con mayor cantidad de hogares/personas con alguna necesidad básica insatisfecha (NBI)?
    2. Según nivel educativo, ¿Cómo se distribuye el % de personas con un nivel educativo mayor a secundario completo? (cods 6,7,8 var P09 Población) ¿Y con al menos secundario completo?
    

Primero, se calculó cuantas personas tienen al menos alguna necesidad básica insatisfecha según los criterios del CENSO 2010


In [ ]:
#Filtramos la base del censo de caba por la cantidad de personas que tienen alguna necesidad básica insatisfecha(cod 1)
#Luego, generamos una nueva tabla donde agrupamos por comuna y contabilizamos la cantidad de personas que cumplen con esa condición.
#La tabla se ordenó de acuerdo a las comunas que tenían mayor cantidad de personas con algún NBI. Reseteamos el índice para que LOCAL sea nuestra primera columna y creamos la columna conteo_ALGUNBI

CANT_personas_nbiP = censo_caba[censo_caba['ALGUNBI'] == 1].groupby('LOCAL')['PERSONA_REF_ID'].count().sort_values(ascending=False).reset_index(name='conteo_ALGUNBI')
pd.DataFrame(CANT_personas_nbiP)

#Calculamos el % del total de personas con alguna NBI en caba que representa cada comuna. 
CANT_personas_nbiP['P_de_poblacion_total_con_algun_NBI'] = ((CANT_personas_nbiP['conteo_ALGUNBI'] / CANT_personas_nbiP['conteo_ALGUNBI'].sum()) * 100).round(1).astype(str) + '%'

#De la base del censo, calculamos cuantas personas existen por comuna en la base, contabilizando la columna PERSONA_REF_ID. creamos una nueva tabla que agrupara por COMUNA estos datos.
poblacion_por_comuna = censo_caba.groupby('LOCAL')['PERSONA_REF_ID'].count().reset_index(name='Poblacion_de_comuna')

#Mergeamos nuestra primera tabla donde teniamos los datos de las necesidades básicas insatisfechas con la tabla de población por comuna.
CANT_personas_nbiP = pd.merge(CANT_personas_nbiP, poblacion_por_comuna, on='LOCAL')

#Una vez hecho esto, calculamos el % que representaba la cantidad de personas con NBI para la comuna y generamos una nueva columna
CANT_personas_nbiP['P_segun_poblacion_de_comuna'] = ((CANT_personas_nbiP['conteo_ALGUNBI'] / CANT_personas_nbiP['Poblacion_de_comuna']) * 100).round(1)
Total_NBI_caba = CANT_personas_nbiP['conteo_ALGUNBI'].sum()

print(f'Total de personas con al menos una necesidad básica insatisfecha en CABA: {Total_NBI_caba}')
pd.DataFrame(CANT_personas_nbiP)

: 

Luego creamos un mapa para visualizar cuales son las comunas que tienen mayor porcentaje de personas con al menos una necesidad básica insatisfecha en sus hogares.


In [ ]:
# Crear GeoDataFrame con los datos de de la base df_caba, donde contamos con las coordenadas de cada distrito.
gdf = gpd.GeoDataFrame(df_caba, geometry=gpd.GeoSeries.from_wkt(df_caba['WKT']))

#forzamos que comuna sea un valor de tipo int para generar luego un join, lo mismo hicimos con la tabla que queríamos vincular "CANT personas nbiP"
gdf['COMUNA'] = gdf['COMUNA'].astype(int)
CANT_personas_nbiP['LOCAL'] = CANT_personas_nbiP['LOCAL'].astype(int)

#Hicimos un left join entre gdf (con los datos de las comunas de caba) y "CANT personas nbiP", con los datos de necesidades básicas insatisfechas
gdf = gdf.join(CANT_personas_nbiP.set_index('LOCAL'), on='COMUNA', how='left')

# Convertir la columna P_segun_poblacion_de_comuna a valores numéricos, redondeamos los datos y los convertimos a float
gdf['P_segun_poblacion_de_comuna'] = gdf['P_segun_poblacion_de_comuna'].round(1).astype(float)

# Dissolver por COMUNA y calcular el promedio de "% según población de comuna"
gdf_dissolved = gdf.dissolve(by='COMUNA', aggfunc='mean')

# Crear el gráfico
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Cambiar colores y agregar etiquetas
gdf_dissolved.plot(column='P_segun_poblacion_de_comuna', ax=ax, legend=True,
                   legend_kwds={'label': "% de con al menos un NBI en la comuna",
                                'orientation': "horizontal"}, cmap='Reds', 
                   vmin=gdf['P_segun_poblacion_de_comuna'].min(), vmax=gdf['P_segun_poblacion_de_comuna'].max(),
                   edgecolor='black', linewidth=0.5)

# Agregar etiquetas a las comunas
for x, y, label in zip(gdf_dissolved.geometry.centroid.x, gdf_dissolved.geometry.centroid.y, gdf_dissolved.index):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points", fontsize=8, ha='center', va='center', color='black')


# Añadir título y etiquetas
ax.set_title('Distribución - Porcentaje de personas con NBI en la comuna', fontdict={'fontsize': '15', 'fontweight': '3'})
ax.set_xlabel(None)
ax.set_ylabel(None)
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

plt.show()

: 

Realizamos un análisis de la población con un nivel educativo de al menos el secundario completo en 2010, por comuna. Para ello, tomamos los códigos 5,6,7,8 de la variable P09 de la tabla de población

In [ ]:
CANT_personas_ES = censo_caba[censo_caba['P09'].isin([5,6,7,8])].groupby('LOCAL')['PERSONA_REF_ID'].count().sort_values(ascending=False).reset_index(name='conteo_personas_secundario')
pd.DataFrame(CANT_personas_ES)

CANT_personas_ES['Porcentaje que representa de la población total'] = ((CANT_personas_ES['conteo_personas_secundario'] / CANT_personas_ES['conteo_personas_secundario'].sum()) * 100).round(1).astype(str) + '%'

poblacion_por_comuna = censo_caba.groupby('LOCAL')['PERSONA_REF_ID'].count().reset_index(name='Población de comuna')
poblacion_por_comuna_m18 = censo_caba[censo_caba['P03'] >= 18].groupby('LOCAL')['PERSONA_REF_ID'].count().reset_index(name='Población de comuna_m18')

merged_df = pd.merge(CANT_personas_ES, poblacion_por_comuna, on='LOCAL')

# Segundo merge entre merged_df y poblacion_por_comuna_m18
CANT_personas_ES = pd.merge(merged_df, poblacion_por_comuna_m18, on='LOCAL')

CANT_personas_ES['P_segun_poblacion_de_comuna'] = ((CANT_personas_ES['conteo_personas_secundario'] / CANT_personas_ES['Población de comuna_m18']) * 100).round(1)


CANT_personas_ES.sort_values(by='P_segun_poblacion_de_comuna', ascending=False)

: 

Luego, realizamos el mismo gráfico que aplicamos en el análisis de los casos con al menos un NBI, pero para el porcentaje de personas con el secundario o más del secundario completo


In [ ]:
# Crear GeoDataFrame con geometría y datos de censo
gdf2 = gpd.GeoDataFrame(df_caba, geometry=gpd.GeoSeries.from_wkt(df_caba['WKT']))
gdf2['COMUNA'] = gdf2['COMUNA'].astype(int)
CANT_personas_ES['LOCAL'] = CANT_personas_ES['LOCAL'].astype(int)
gdf2 = gdf2.join(CANT_personas_ES.set_index('LOCAL'), on='COMUNA', how='left')

# Convertir la columna a valores numéricos, redondear y convertir a float
gdf2['P_segun_poblacion_de_comuna'] = gdf2['P_segun_poblacion_de_comuna'].round(1).astype(float)

# Dissolver por COMUNA y calcular el promedio de "% según población de comuna"
gdf2_dissolved = gdf2.dissolve(by='COMUNA', aggfunc='mean')

# Crear el gráfico
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Cambiar colores y agregar etiquetas
gdf2_dissolved.plot(column='P_segun_poblacion_de_comuna', ax=ax, legend=True,
                   legend_kwds={'label': "Porcentaje de personas con al menos secundario completo en la comuna",
                                'orientation': "horizontal"}, cmap='Blues', 
                   vmin=gdf2['P_segun_poblacion_de_comuna'].min(), vmax=gdf2['P_segun_poblacion_de_comuna'].max(),
                   edgecolor='black', linewidth=0.5)

# Agregar etiquetas a las comunas
for x, y, label in zip(gdf2_dissolved.geometry.centroid.x, gdf2_dissolved.geometry.centroid.y, gdf2_dissolved.index):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points", fontsize=8, ha='center', va='center', color='black')


# Añadir título y etiquetas
ax.set_title('Distribución - Porcentaje de personas con al menos secundario completo en la comuna', fontdict={'fontsize': '15', 'fontweight': '3'})
ax.set_xlabel(None)
ax.set_ylabel(None)
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

plt.show()

: 

## Análisis de los resultados en CABA

Para producir un análisis de los resultaods electorales en CABA iniciamos reduciendo la base de datos inicial para que coincida con el recorte geográfico requerido. 

In [ ]:
print("Tabla de resultados elecciones generales 2023- CABA")
df_resultados_caba = df_resultados[df_resultados['distrito_id'] == 1]
cant_votos_total_caba = df_resultados_caba['votos_cantidad'].sum()
print(f'Estructura: {df_resultados_caba.shape}')
print(f'Cantidad de votos totales: {cant_votos_total_caba}')
df_resultados_caba.head()

: 

### Revisión de las tablas

Analizamos los valores de la base importada para comprender la distribución de los valores en ella, con el siguiente código.

In [ ]:
for columna in df_resultados_caba.columns:
    print(f"Opciones únicas en la columna {columna}:")
    print(df_resultados_caba[columna].unique())
    print("\n") 

: 

Luego, implementamos el mismo análisis para el resto de las tablas sobre elecciones en 2023.

In [ ]:
print("Tabla circuitos electorales en CABA")
for columna in df_caba.columns:
    print(f"Opciones únicas en la columna {columna}:")
    print(df_caba[columna].unique())
    print("\n") 

: 

In [ ]:
print("Tabla colores para mapas")
for columna in df_colores.columns:
    print(f"Opciones únicas en la columna {columna}:")
    print(df_colores[columna].unique())
    print("\n")  

: 

In [ ]:
print("Tabla ambitos generales")
for columna in df_ambitos.columns:
    print(f"Opciones únicas en la columna {columna}:")
    print(df_ambitos[columna].unique())
    print("\n") 

: 

### Vinculación entre tablas - Bases elecciones


Comenzamos haciendo un merge entre dos tablas requeridas para el análisis de los resultados electorales: la primera, la base de elecciones filtrada para Ciudad de Buenos Aires (df_resultados_caba), la segunda, la base sobre los circuitos electorales y sus características en este mismo distrito (df_caba). Ambas se relacionan a partir de la columna "seccion_id" (df_resultados_caba) y la columna "comuna" (df_caba) cuyos valores son los ids asignados a las comunas de la ciudad.

In [ ]:
df_resultados_caba['seccion_id'] = df_resultados_caba['seccion_id'].astype(int) #forzamos el formato int para que el id se reconozca como número entero
df_caba['COMUNA'] = df_caba['COMUNA'].astype(int) #forzamos el formato int para que el id se reconozca como número entero


: 

In [ ]:
#Generamos un merge que tenga como tabla principal la base de resultados de caba y agregue las columnas de la base de circuitos electorales de acuerdo a las valores coincidentes con la primera.
df_combinado = pd.merge(df_resultados_caba, df_caba, left_on='seccion_id', right_on='COMUNA', how='left')

print('\n')
print("Tabla Combinada")
print(f'Estructura: {df_combinado.shape}')
df_combinado.head()



: 

In [ ]:
df_resultadosCaba = df_resultados[(df_resultados['distrito_id'] == 1) & 
                                  (df_resultados['cargo_nombre'] == "PRESIDENTE Y VICE")]

# Seleccionar solo las columnas requeridas
df_resultadosCaba = df_resultadosCaba[['seccion_id', 'circuito_id', 
                                       'mesa_id', 'agrupacion_nombre', 
                                       'votos_cantidad']]

# Restablecer el índice para que comience en 0
df_resultadosCaba.reset_index(drop=True, inplace=True)

# Mostrar las primeras filas del nuevo DataFrame para verificar
print(df_resultadosCaba.head(2))

: 

In [ ]:
df_resultados_agrupado = df_resultadosCaba.groupby(['seccion_id', 'agrupacion_nombre'], as_index=False)['votos_cantidad'].sum()
df_resultados_agrupado

: 

Ganadores por comuna en las elecciones generales de 2023 - Por partido

In [ ]:
# Encontrar el índice del máximo en la columna 'votos_cantidad' dentro de cada grupo
idx_max_votos = df_resultados_agrupado.groupby('seccion_id')['votos_cantidad'].idxmax()

# Obtener el DataFrame de los ganadores
df_ganadores_seccion = df_resultados_agrupado.loc[idx_max_votos]

# Mostrar las primeras filas del DataFrame de los ganadores
print(df_ganadores_seccion.head(15))

: 

### Análisis conjunto de necesidades básicas insatisfechas en 2010 y ganadores elecciones generales 2023

#### Mapa combinado

In [ ]:
import seaborn as sns

# Crear GeoDataFrame con geometría y datos de censo
gdf3 = gpd.GeoDataFrame(df_caba, geometry=gpd.GeoSeries.from_wkt(df_caba['WKT']))
gdf3['COMUNA'] = gdf3['COMUNA'].astype(int)
CANT_personas_nbiP['LOCAL'] = CANT_personas_nbiP['LOCAL'].astype(int)
gdf3 = gdf3.join(CANT_personas_nbiP.set_index('LOCAL'), on='COMUNA', how='left')
gdf3 = gdf3.join(df_ganadores_seccion.set_index('seccion_id'), on='COMUNA', how='left')

# Convertir la columna a valores numéricos, redondear y convertir a float
gdf3['P_segun_poblacion_de_comuna'] = gdf3['P_segun_poblacion_de_comuna'].round(1).astype(float)

# Dissolver por COMUNA y calcular el promedio de "% según población de comuna"
gdf3_dissolved = gdf3.dissolve(by='COMUNA', aggfunc='mean')

# Crear el gráfico
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Cambiar colores y agregar etiquetas
gdf3_dissolved.plot(column='P_segun_poblacion_de_comuna', ax=ax, legend=True,
                    legend_kwds={'label': "% de con al menos un NBI en la comuna",
                                 'orientation': "horizontal"}, cmap='Reds', 
                    vmin=gdf3['P_segun_poblacion_de_comuna'].min(), vmax=gdf3['P_segun_poblacion_de_comuna'].max(),
                    edgecolor='black', linewidth=0.5)

# Agregar etiquetas a las comunas
for x, y, label in zip(gdf3_dissolved.geometry.centroid.x, gdf2_dissolved.geometry.centroid.y, gdf3_dissolved.index):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points", fontsize=8, ha='center', va='center', color='black')

# Agregar puntos para los ganadores de cada comuna
for idx, row in gdf3.iterrows():
    color = 'yellow' if row['agrupacion_nombre'] == 'JUNTOS POR EL CAMBIO' else 'blue'
    size = row['votos_cantidad'] / gdf3.groupby('agrupacion_nombre')['votos_cantidad'].transform('max').max() * 200
    centroid = gdf3_dissolved.geometry.centroid.loc[gdf3_dissolved.index == row['COMUNA']].squeeze()
    ax.scatter(centroid.x + (-0.007), centroid.y + 0.005, color=color, s=size, alpha=0.7, edgecolors='black', linewidth=0.5)
    
# Usar seaborn para mostrar leyenda
legend_labels = {'JUNTOS POR EL CAMBIO': 'yellow', 'UNION POR LA PATRIA': 'blue'}
for label, color in legend_labels.items():
    ax.scatter([], [], color=color, label=label)

# Añadir título y etiquetas
ax.set_title('% personas con algún NBI y ganadores de las elecciones generales 2023', fontdict={'fontsize': '15', 'fontweight': '3'})
ax.set_xlabel(None)
ax.set_ylabel(None)
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

# Ajustar posición de la leyenda
ax.legend(loc='upper left', bbox_to_anchor=(0.8, 1))
plt.show()


: 

Para realizar una correlacion entre las variables analizadas, generamos una tabla nueva que cuenta con los datos de cantidad de votos por agrupación política, cantidad de personas con alguna NBI y cantidad de personas con estudios secundarios o superiores.

In [ ]:
df_resultados_caba_pivot = df_resultados_agrupado.pivot_table(index='seccion_id',columns='agrupacion_nombre',values='votos_cantidad')

: 

In [ ]:
df_total = df_resultados_caba_pivot.join(CANT_personas_nbiP.set_index('LOCAL')['conteo_ALGUNBI'], on='seccion_id', how='left')
df_total = df_total.join(CANT_personas_ES.set_index('LOCAL')['conteo_personas_secundario'], on='seccion_id', how='left')
df_total = df_total.reset_index()
df_total

: 

#### Correlación Juntos por el Cambio y variables sociodemográficas

In [ ]:
print("Correlación Juntos por el Cambio y cantidad de personas con alguna NBI")
print("\n")

print('Correlación Pearson: ', df_total['JUNTOS POR EL CAMBIO'].corr(df_total['conteo_ALGUNBI'], method='pearson'))
print('Correlación spearman: ', df_total['JUNTOS POR EL CAMBIO'].corr(df_total['conteo_ALGUNBI'], method='spearman'))
print('Correlación kendall: ', df_total['JUNTOS POR EL CAMBIO'].corr(df_total['conteo_ALGUNBI'], method='kendall'))

print("\n")
print("Correlación Juntos por el Cambio y cantidad de personas con al menos estudios secundarios")
print("\n")

print('Correlación Pearson: ', df_total['JUNTOS POR EL CAMBIO'].corr(df_total['conteo_personas_secundario'], method='pearson'))
print('Correlación spearman: ', df_total['JUNTOS POR EL CAMBIO'].corr(df_total['conteo_personas_secundario'], method='spearman'))
print('Correlación kendall: ', df_total['JUNTOS POR EL CAMBIO'].corr(df_total['conteo_personas_secundario'], method='kendall'))

: 

#### Correlación La Libertad Avanza y variables sociodemográficas

In [ ]:
print("Correlación La Libertad Avanza y cantidad de personas con alguna NBI")
print("\n")

print('Correlación Pearson: ', df_total['LA LIBERTAD AVANZA'].corr(df_total['conteo_ALGUNBI'], method='pearson'))
print('Correlación spearman: ', df_total['LA LIBERTAD AVANZA'].corr(df_total['conteo_ALGUNBI'], method='spearman'))
print('Correlación kendall: ', df_total['LA LIBERTAD AVANZA'].corr(df_total['conteo_ALGUNBI'], method='kendall'))

print("\n")
print("Correlación La Libertad Avanza y cantidad de personas con al menos estudios secundarios")
print("\n")

print('Correlación Pearson: ', df_total['LA LIBERTAD AVANZA'].corr(df_total['conteo_personas_secundario'], method='pearson'))
print('Correlación spearman: ', df_total['LA LIBERTAD AVANZA'].corr(df_total['conteo_personas_secundario'], method='spearman'))
print('Correlación kendall: ', df_total['LA LIBERTAD AVANZA'].corr(df_total['conteo_personas_secundario'], method='kendall'))

: 

#### Correlación Unión por la Patria y variables sociodemográficas

In [ ]:
print("Correlación Unión por la Patria y cantidad de personas con alguna NBI")
print("\n")

print('Correlación Pearson: ', df_total['UNION POR LA PATRIA'].corr(df_total['conteo_ALGUNBI'], method='pearson'))
print('Correlación spearman: ', df_total['UNION POR LA PATRIA'].corr(df_total['conteo_ALGUNBI'], method='spearman'))
print('Correlación kendall: ', df_total['UNION POR LA PATRIA'].corr(df_total['conteo_ALGUNBI'], method='kendall'))

print("\n")
print("Correlación Unión por la Patria y cantidad de personas con al menos estudios secundarios")
print("\n")

print('Correlación Pearson: ', df_total['UNION POR LA PATRIA'].corr(df_total['conteo_personas_secundario'], method='pearson'))
print('Correlación spearman: ', df_total['UNION POR LA PATRIA'].corr(df_total['conteo_personas_secundario'], method='spearman'))
print('Correlación kendall: ', df_total['UNION POR LA PATRIA'].corr(df_total['conteo_personas_secundario'], method='kendall'))

: 

### Relación con la participación electoral

In [ ]:
df_caba.head(4)
df_caba.groupby('COMUNA')['TOTAL'].sum().reset_index(name='Votantes totales')
df_caba2 = pd.DataFrame(df_caba.groupby('COMUNA')['TOTAL'].sum().reset_index(name='Votantes totales'))

: 

In [ ]:
df_resultadosCaba3 = df_resultados[(df_resultados['distrito_id'] == 1)]
df_resultadosCaba3 = df_resultadosCaba3[['seccion_id', 'circuito_id', 
                                       'mesa_id', 'agrupacion_nombre', 
                                       'votos_cantidad']]
df_resultadosCaba3.reset_index(drop=True, inplace=True)


df_resultadosCaba3.groupby('seccion_id')['votos_cantidad'].sum().reset_index(name='Votos emitidos')
df_resultadosCaba2 =pd.DataFrame(df_resultadosCaba.groupby('seccion_id')['votos_cantidad'].sum().reset_index(name='Votos emitidos'))
df_resultadosCaba2 = pd.merge(df_caba2.set_index('COMUNA'), df_resultadosCaba2.set_index('seccion_id'), how='left', left_index=True, right_index=True)
df_resultadosCaba2.reset_index(inplace=True)
df_resultadosCaba2['Participacion_por_comuna'] = (df_resultadosCaba2['Votos emitidos'] / df_resultadosCaba2['Votantes totales'] *100).round(1).astype('float')

participacion_total_caba = (df_resultadosCaba2['Votos emitidos'].sum() / df_resultadosCaba2['Votantes totales'].sum() *100).round(1).astype('float')
print(f'Participación total en CABA: {participacion_total_caba}')
votantes_totales_caba = df_resultadosCaba2['Votantes totales'].sum()
print(f'Cantidad de votantes en CABA: {votantes_totales_caba}')
df_resultadosCaba2


: 

In [ ]:
gdf5 = gpd.GeoDataFrame(df_caba, geometry=gpd.GeoSeries.from_wkt(df_caba['WKT']))
gdf5 = gdf5.join(df_resultadosCaba2.set_index('COMUNA'), on='COMUNA', how='left')

# Dissolver por COMUNA y calcular el promedio de "% según población de comuna"
gdf5_dissolved = gdf5.dissolve(by='COMUNA', aggfunc='mean')

# Crear el gráfico
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Cambiar colores y agregar etiquetas
gdf5_dissolved.plot(column='Participacion_por_comuna', ax=ax, legend=True,
                   legend_kwds={'label': "%Participación por comuna",
                                'orientation': "vertical"}, cmap='Blues_r', 
                   edgecolor='black', linewidth=0.5)

# Agregar etiquetas a las comunas
for x, y, label in zip(gdf5_dissolved.geometry.centroid.x, gdf5_dissolved.geometry.centroid.y, gdf5_dissolved.index):
    ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points", fontsize=8, ha='center', va='center', color='black')


# Añadir título y etiquetas
ax.set_title('%Participación por comuna', fontdict={'fontsize': '15', 'fontweight': '3'})
ax.set_xlabel(None)
ax.set_ylabel(None)
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

plt.show()

: 

#### Correlacion participación y votos


In [ ]:
df_total2 = pd.merge(df_total, df_resultadosCaba2, left_on='seccion_id', right_on='COMUNA', how='left')
df_total2

: 

#### Correlación partido y cantidad de votos


In [ ]:
def calcular_correlaciones(df_total2, columna_votos='Votos emitidos'):
    # Filtrar solo las columnas numéricas para calcular correlaciones
    columnas = ['FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD', 'HACEMOS POR NUESTRO PAIS', 'JUNTOS POR EL CAMBIO', 'LA LIBERTAD AVANZA', 'UNION POR LA PATRIA']
    
    # Iterar sobre las columnas y calcular la correlación con 'cantidad de votos'
    for columna in columnas:
        correlacion = df_total2[columna].corr(df_total2[columna_votos], method='pearson')
        print(f"La correlación entre {columna} y {columna_votos} es: {correlacion.round(2)}")

calcular_correlaciones(df_total2, )


: 

In [ ]:
calcular_correlaciones(df_total2, 'Participacion_por_comuna')

: 

## Insights de los resultados

In [ ]:
import scipy.stats as stats


def sample_size(population_size, confidence_level, margin_of_error):
    z = stats.norm.ppf(confidence_level + (1 - confidence_level) / 2)
    return (2 * z**2 * population_size) / (margin_of_error**2)

sample_size(2497035, 0.95, 0.05)

: 

: 